In [ ]:
import os, sys

In [58]:
from functools import wraps
class RerunOnError(Exception):
    pass

def rerun(function):
    @wraps(function)
    def rerun_wrapper(*args, **kwargs):
        previously_raised = None
        NO_OF_TRIES = 3
        for _ in range(NO_OF_TRIES):
            try:
                return function(*args, **kwargs)
            except RerunOnException as exp:
                logger.info("Rerun%s",function.__qualname__)
                previously_raised = exp
        raise previously_raised
    return rerun_wrapper


In [59]:
@rerun
def schedule_task():
    """schedules a task, throws an Exception is resources are busy"""
    try:
        print("hello")
        int("damn")
        return 0
    except:
        print("world")
        return 1


In [60]:
schedule_task()

hello
world


1

In [61]:
# 1.1.1	Décoration of functions

import time
from functools import wraps

def rerun(ExceptionTuple, tries=4, delay=3, backoff=2):
    def retry(func):
        @wraps(func)
        def run_again(*args, **kwargs):
            maxtries, maxdelay = tries, delay
            while maxtries > 1:
                try:
                    return func(*args, **kwargs)
                except ExceptionTuple as e:
                    print(f"Failed {str(e)}! Retrying in {maxdelay} seconds...")  
                    time.sleep(maxdelay)
                    maxtries -= 1
                    maxdelay *= backoff
            return func(*args, **kwargs)
        return run_again  # true decorator
    return retry

In [62]:
@rerun(Exception, tries=4)
def schedule_task():
    """schedules a task, throws an Exception when resources are busy"""
    int("damn")
    print("hello")
    int("damn")
    return 0

In [63]:
schedule_task()

Failed invalid literal for int() with base 10: 'damn'! Retrying in 3 seconds...
Failed invalid literal for int() with base 10: 'damn'! Retrying in 6 seconds...
Failed invalid literal for int() with base 10: 'damn'! Retrying in 12 seconds...


ValueError: invalid literal for int() with base 10: 'damn'

In [ ]:
# 1.1.2	Décoration of classes

class SerializeMessages:
    def __init__(self, Message):
        self.message= message

    def serialize(self) ->dict:
        return {
                   "login_name": self.message.login_name,
                   "access_key": "**redacted**",
                   "connection": self.message.connection,
                   "timestamp": self.message.timestamp.strftime("%Y-%m-%d%H:%M"),
        }

class LoginMessage:
    MESSAGE_SERIALIZER = SerializeMessages
    def __init__(self, login_name, access_key, connection,timestamp):
        self.login_name = login_name
        self.access_key = access_key
        self.connection = connection
        self.timestamp = timestamp

    def serialize(self) -> dict:
        return self.MESSAGE_SERIALIZER(self).serialize()


In [64]:
# Define the serializer class

class SerializeMessages:
    def __init__(self, message_flds: dict) -> None:
         self.message_flds = message_flds

    def serialize(self, message) ->dict:
        return {
            field: transformation(getattr(message, field))
            for field, transformation in
            self.message_fields.items()
        }

# MsgSerialization class
class MsgSerialization:
    def __init__(self, **trnsfrm):
        self.serialzr = SerializeMessages(trnsfrm)

    def __call__(self, message_class):
        def function_serializer(message_instance):
             return self.serialzr.serialize(message_instance)
        message_class.serialize = function_serializer
        return message_class


In [65]:
from datetime import datetime

# Define the utility methods
def obfuscate_passkey(passkey) -> str:
    return "**redacted**"

def repr_timestamp(timestamp: datetime) -> str:
    return timestamp.strftime("%Y-%m-%d %H:%M")

def nochange(field_name):
    return field_name

# Applying the decorator to the class 
@MsgSerialization(
    login_name=nochange,
    passkey=obfuscate_passkey,
    connection=nochange,
    timestamp=repr_timestamp,
)
class LoginMessage:
    def __init__(self, login_name, passkey, connection, timestamp):
        self.login_name= login_name
        self.passkey= passkey
        self.connection = connection
        self.timestamp = timestamp


In [66]:
# Using dataclasses you could make the implementation of decorators even simpler

from dataclasses import dataclass
import datetime

@MsgSerialization(
login_name='nochange',
passkey='obfuscate_passkey',
timestamp='repr_timestamp',
)

@dataclass
class LoginMessage:
    login_name: str
    passkey: str
    passkey: str
    timestamp: datetime.datetime


In [67]:
# 1.1.3	Decoration of other constructs - example, generators

from time import time
import functools

def log_method_calls(function):
    @functools.wraps(function)
    def logging_wrapper(*args, **kwargs):
        t_begin = time()
        value = yield from function(*args, **kwargs)
        t_end = time()
        duration = t_end - t_begin
        print('Function %s took %f' % (function.__name__, duration))
        return value
    return logging_wrapper


In [68]:
from time import time, sleep

@log_method_calls
def demo():
    for index in range(5):
        sleep(index)
        yield index

In [69]:
tuple(demo())

Function demo took 10.019515


(0, 1, 2, 3, 4)

In [70]:
# 1.1.5	Evaluation of multiple décorators

def salutation(function):
    def add_salutation_wrap():
        salutation = function()
        return (" ".join([salutation, "Stephen Hawking!"]))
    return add_salutation_wrap

def change_case(function):
    def to_upper():
        case_txt = function()
        if not isinstance(case_txt, str):
            raiseTypeError("Not a string type")
        return case_txt.upper()
    return to_upper


In [71]:
@change_case
@salutation
def get_salutation():
    return "Hello"

In [72]:
get_salutation()

'HELLO STEPHEN HAWKING!'

In [73]:
from datetime import datetime
def enable_logs(function):
    def enable_logs_wrapper(*args, **kwargs):
        print(function.__name__ + " was invoked at " +  str(datetime.now()))
        return function(*args, **kwargs)
    return enable_logs_wrapper

@enable_logs
def get_square(number):
    return number * number


In [74]:
square = get_square(9)

get_square was invoked at 2021-05-08 18:45:04.288211


In [75]:
print(get_square.__name__)

enable_logs_wrapper


In [76]:
# 1.1.6	Using functools Library for Decorators

from datetime import datetime
from functools import wraps

def enable_logs(function):
    @wraps(function)     #--- use the functools API to wrap 
    def enable_logs_wrapper(*args, **kwargs):
        print(function.__name__ + " was invoked at " +  str(datetime.now()))
        return function(*args, **kwargs)
    return enable_logs_wrapper

@enable_logs
def eval_expression(number):
    """a computing function"""
    return number + number * number

print(eval_expression.__name__)    # prints 'eval_expression'
print(eval_expression.__doc__)     # prints 'a computing function'


eval_expression
a computing function


In [77]:
# a simple third-party library called decorator that houses useful functionality 
# for a more robust definition and use of decorators.

from decorator import decorator

@decorator
def call_log(function, *args, **kwargs):
    kwords = ''
    for key in sorted(kwargs):
        kwords = ', '.join('%r: %r' % (key, kwargs[k])) 
    print("Function %s called! Arguments %s, {%s}" % (function.__name__, args, kwords))
    return function(*args, **kwargs)


@call_log
def random_function(): 
    pass

random_function()

Function random_function called! Arguments (), {}


In [ ]:
# Implementation for creating decorators for the class methods

import requests

def try_again(times=3, pause=10):
    def retry(function):
        @wraps(function)
        def run_again(*args, **kwargs):
            for try_num in times:
                function(*args, **kwargs)
                time.sleep(pause)
        return run_again
    return retry


class DownloadFromRemote:
    def __init__(self, remote_url, hdr_string):
        self.remote_url = remote_url
        self.hdr = hdr_string

    @try_again(times=4, pause=5)
    def request_file(self):
        try:
            rsp = requests.get(self.remote_url, self.hdr)
            if rsp.status_code in (429, 500, 502, 503):
                pass # Add custom handling logic
        except Exception as err:
            raise FailedRequest("Server connection failed!")
        return rsp

In [78]:
# 1.1.7	Stateful Decorators - Maintaining the state within the decorator

import functools

def invoke_count(function):
    @functools.wraps(function)
    def calls_wrapper(*args, **kwargs):
        calls_wrapper.count_of_calls += 1
        print (f"CallNo. {calls_wrapper.count_of_calls}")
        return function(*args, **kwargs)
    calls_wrapper.count_of_calls = 0
    return calls_wrapper

@invoke_count
def hello_world():
    print("Hello to you, too!")


In [79]:
hello_world()

CallNo. 1
Hello to you, too!


In [80]:
hello_world()

CallNo. 2
Hello to you, too!


In [81]:
hello_world()

CallNo. 3
Hello to you, too!


In [82]:
hello_world.count_of_calls

3

In [83]:
# 1.1.8	Creating Singletons with Décorators

import functools

def make_singleton(cls):
    @functools.wraps(cls)
    def sgl_wrapper(*args, **kwargs):
        if not sgl_wrapper.object:
            sgl_wrapper.object = cls(*args, **kwargs)
        return sgl_wrapper.object
    sgl_wrapper.object = None
    return sgl_wrapper

@make_singleton
class DeltaStategies:
    # Some Logic
    pass


In [84]:
strat_one = DeltaStategies()
strat_two = DeltaStategies()

In [85]:
id(strat_one)

1722961249472

In [86]:
strat_one is strat_two

True

In [ ]:
# 1.1.9	Caching Function Return Values – Memoisation

@count_calls # Define a custom count_calls to try this out!
def get_fibo_seies(start_int):
    if start_int< 2:
        return start_int
    return get_fibo_series(start_int - 1) + get_fibo_series(start_int - 2)

In [ ]:
# 1.1.9	Caching Function Return Values – Memoisation

import functools

def enable_caching(function):
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        key_for_cache = args + tuple(kwargs.items())
        if key_for_cache not in wrapper.cache:
            wrapper.cache[key_for_cache] = function(*args, **kwargs)
        return wrapper.cache[key_for_cache]
    wrapper.cache = dict()
    return wrapper

@enable_caching
@count_calls
def get_fibo_series(start_int):
    if start_int< 2:
        return start_int
    return get_fibo_series(start_int - 1) + get_fibo_series(start_int - 2)

In [88]:
# 1.1.9	Caching Function Return Values – Memoisation

import functools

@functools.lru_cache(maxsize=5)
def get_fibo_series(start_int):
    print(f"Getting {start_int}th value")
    if start_int< 2:
        return start_int
    return get_fibo_series(start_int - 1) + \
get_fibo_series(start_int - 2)

In [89]:
get_fibo_series(10)

Getting 10th value
Getting 9th value
Getting 8th value
Getting 7th value
Getting 6th value
Getting 5th value
Getting 4th value
Getting 3th value
Getting 2th value
Getting 1th value
Getting 0th value


55

In [90]:
get_fibo_series(7)

13

In [91]:
get_fibo_series(5)

Getting 5th value
Getting 4th value
Getting 3th value
Getting 2th value
Getting 1th value
Getting 0th value


5

In [92]:
get_fibo_series(7)

Getting 7th value
Getting 6th value


13

In [93]:
get_fibo_series(5)

5

In [94]:
get_fibo_series.cache_info()

CacheInfo(hits=16, misses=19, maxsize=5, currsize=5)

In [ ]:
# 1.1.11	Preserving data about the original wrapped object

# wrap_decor_2.py
def log_tracer(func):
    @wraps(func)
    def wrapped(*args, **kwargs):
        logger.info("running %s", func.__qualname__)
        return func(*args, **kwargs)
    return wrapped

In [ ]:
import time
def func_trace_wrong(method):
    print("Execute method %s", method)
    exec_begin_time = time.time()

    @functools.wraps(method)
    def wrapped(*args, **kwargs):
        output = method(*args, **kwargs)
        print(
            "method takes %.2fs",
            time.time() - exec_begin_time
        )
        return output
    return wrapped


In [ ]:
@func_trace_wrong
def slowdown(callback, time_secs=0):
    time.sleep(time_secs)
    return callback()

In [ ]:
# Every successive call also finds the difference from current time to the import time and not the run start.

def func_trace(method):
    @functools.wraps(method)
    def wrapped(*args, **kwargs):
        print("Execute method %s", method.__qualname__)
        exec_begin_time = time.time()
        output = method(*args, **kwargs)
        print(
            "method %s takes %.2fs",
            method.__qualname__,
            time.time() - start_time
        )
        return result
    return wrapped


In [95]:
# 1.1.13	Advantages of decorators with side-effects

EXPOSED_EVENT_SET = {}

def register_for_use(myEvent):
    EXPOSED_EVENT_SET[myEvent.__name__] = myEvent
    return myEvent

class Event:
    class customEvent:
        TYPE = "specificEvent"

    @register_for_use
    class LoginUserHandler(customEvent):
        """ Executed when the user is logging in to the system """
        pass

    @register_for_use
    class LogoutEventHandler(customEvent):
        """Invoked when the logout operation is initiated/completed """
        pass


In [96]:
EXPOSED_EVENT_SET

{'LoginUserHandler': __main__.Event.LoginUserHandler,
 'LogoutEventHandler': __main__.Event.LogoutEventHandler}

In [97]:
# 1.1.14	Create decorators that work for multiple object types.

import logging
from functools import wraps

# Logger instances
logger = logging.getLogger(__name__)

class DatabaseAdapter:
    def __init__(self, databaseParameters):
        self.databaseParameters= databaseParameters

    def run_query(self, query_string):
        return(f"Running {query_string} with params {self.databaseParameters}")

    # Define the decorator
    def use_database_driver(func):
        """ Creates and returns a driver object from the param string """
        @wraps(func)
        def wrapped(databaseParameters):
            return func(DatabaseAdapter(databaseParameters))
        return wrapped

    @use_database_driver
    def execute_query(db_driver):
        return db_driver.run_query("some_random_function")


In [98]:
# Making decorators handle cases with 'self'

from types import MethodType
from functools import wraps

class use_database_driver:
    """ Creates and returns a driver object from the param string “””"""
    def __init__(self, func):
        self.func = func
        wraps(self.func)(self)

    def __call__(self, connectionString):
        return self.func(DatabaseAdapter(connectionString))

    def __get__(self, obj, owner):
        if obj is None:
            return self
        return self.__class__(MethodType(self.func, obj))


In [99]:
# 1.1.16	Control Execution Rate of Code

import time
import functools

def reduce_execution_rate(_function=None, *, num_seconds=2):    
    def decorator_exec_rate_reduction(function):
        @functools.wraps(function)
        def wrapper_rate_reduction(*args, **kwargs):
            time.sleep(num_seconds)
            return function(*args, **kwargs)
        return wrapper_rate_reduction

    if _function is None:
        return decorator_exec_rate_reduction
    else:
        return decorator_exec_rate_reduction(_function)


In [100]:
@reduce_execution_rate(num_seconds=3)
def clock_to_zero(start_integer):
    if start_integer< 1:
        print("Shoot!")
    else:
        print(start_integer)
        clock_to_zero(start_integer - 1)


In [101]:
clock_to_zero(5)

5
4
3
2
1
Shoot!


In [102]:
# Separation of Concerns in Décorators

import functools
def func_trace(method):
    @functools.wraps(method)
    def wrapper(*args, **kwargs):
            logger.info("Method name %s executing", method.__qualname__)
            run_begin_time= time.time()
            output = method(*args, **kwargs)
            logger.info(
                    "method %s takes %.2fs",
                    method.__qualname__,
                    time.time() - run_begin_time
                   )
            return output
    return wrapper


In [103]:
# smaller decorators that define a unit level of responsibility each, giving the freedom of choice to the end-user

def exec_trace(method):
    @wraps(method)
    def myWrapper(*args, **kwargs):
        logger.info("Method name %s executing", method.__qualname__)
        return method(*kwargs, **kwargs)
    return myWrapper

def time_profiler(method):
    @wraps(method)
    def myWrapper(*args, **kwargs):
        run_begin = time.time()
        output = method(*args, **kwargs)
        logger.info("Method %s takes %.2f", method.__qualname__,
                     time.time() - run_begin)
        return output
    return myWrapper


In [104]:
@time_profiler
@exec_trace
def operation():
    print("Hello World!!")

In [105]:
operation()

Hello World!!


In [106]:
class CustomConMan:
    def __init__(self):
        print("Initiating Object Creation")
        self.someData = 42

    def __enter__(self):
        print("Within the __enter__ block")
        return self

    def __exit__(self, data_type, data, data_traceback, exc_type=None):
        print('Within the __exit__ block')
        if exc_type:
            print(f"data_type: {data_type}")
            print(f"data: {data }")
            print(f"data_traceback: {data_traceback}")


In [107]:
cntx = CustomConMan()

Initiating Object Creation


In [108]:
cntx.someData

42

In [109]:
with cntx as cm:
    print("Within code block")


Within the __enter__ block
Within code block
Within the __exit__ block


In [ ]:
# 1.1.18	Using contextlib for Creating Context Managers

from contextlib import contextmanager

@contextmanager
def file_dump(name_of_file):
    try:
        fl_handle = open(name_of_file, "w")
        yield fl_handle
    finally:
        fl_handle.close()


In [ ]:
with file_dump("time_series.txt") as fl:
    fl.write("Some random data!")

In [ ]:
# 1.1.19	Safe Database Access 

import sqlite3

connection = sqlite3.connect(":memory:")
connection.execute("create table car (id integer primary key,number varchar unique)")
with connection:
    connection.execute("insert into car(number) values (?)", ("KA-4393",))

try:
    with connection:
        connection.execute("insert into car(number) values (?)", ("KA-4393",))
except sqlite3.IntegrityError:
    print("Unable to add the same entry twice.")


In [ ]:
# 1.1.20	Writing Tests

import pytest

def integer_division(num1, num2):
    if isinstance(num1, int) and isintance(num2, int):
        raiseValueError("Please enter integer values!")
    try:
        return num1/num2
    except Exception:
        print("Denominator should not be Zero!")
        raise

with pytest.raises(ValueError):
    integer_division("21", 7)


In [ ]:
# 1.1.21	Resource Sharing In Python

from filelock import FileLock

def update_file_entry(staticticsFile):
    with FileLock(staticticsFile):
        # File is now locked for updation
        perform_operations()


In [110]:
# 1.1.22	Remote Connection Protocol

import socket

class NetworkResourceAccess:
    def __init__(self, hostname, access_port):
        self.hostname = hostname
        self.access_port = access_port

    def __enter__(self):
        self._remoteMachine = socket()
        self._remoteMachine.connect((self.hostname, 
                                     self.access_port))
        return self

    def __exit__(self, exception, return_code, traceback):
        self._remoteMachine.close()

    def receive(self):
        get_data_util()

    def send(self, data):
        send_data_util(data)


In [ ]:
with NetworkResourceAccess(hostname, access_port) as netgear:
    netgear.send(['entryData1', 'entryData2'])
    output = netgear.receive()